In [ ]:
#@title Start up
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

%pip install ffmpeg

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

import numpy as np
import pandas as pd
import pickle
import cv2
import math
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
from IPython.display import HTML

from collections import defaultdict
from sklearn.neighbors import KDTree
from sklearn.neighbors import NearestNeighbors

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Circle, CirclePolygon, Ellipse
from shapely.geometry import Polygon

from matplotlib import rc
rc('animation', html='jshtml')

rootdir = "/content/drive/MyDrive/XAMI-MIMIC"
%matplotlib inline


# show all rows from dataframe
pd.set_option('display.max_rows', 100)
# show all columns from dataframe
pd.set_option('display.max_columns', 100)

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#@title Start Reflacx Variable
%%script echo skipping
import os

reflacx = {}

with os.scandir(rootdir) as dirsList:
    for dirEntry in dirsList:
        if dirEntry.is_dir():
            with os.scandir(dirEntry.path) as dirsList1:
                if any('REFLACX' in entry.name for entry in dirsList1):
                    reflacx[dirEntry.name] = []
                    metadata_path = os.path.join(dirEntry.path, 'REFLACX/metadata.csv')
                    metadata = pd.read_csv(metadata_path)
                    jpg_name = metadata['dicom_id'][0]
                    x_size = metadata['image_size_x'][0]
                    y_size = metadata['image_size_y'][0]
                    with os.scandir(os.path.join(dirEntry.path, 'CXR-JPG')) as jpgFilesList:
                        for jpgFilesEntry in jpgFilesList:
                            if jpgFilesEntry.name.startswith('s'):
                                with os.scandir(jpgFilesEntry.path) as files3:
                                    for filenameEntry in files3:
                                        if filenameEntry.name.startswith(jpg_name):
                                            reflacx[dirEntry.name].append([jpgFilesEntry.name, filenameEntry.name, x_size, y_size])
                    
                    dirsList4 = os.listdir(os.path.join(dirEntry.path, 'REFLACX'))
                    reflacx[dirEntry.name].append(dirsList4[1:])

print(reflacx)


skipping


In [ ]:
#@title Init Variables
SCREEN_RES = [3180, 2160]

FONT_SIZE = 4

to_save_images = "/content/drive/MyDrive/REFLACX-plots"
ignore_labels = ['xmin', 'ymin', 'xmax', 'ymax', 'certainty']

ellipses_labels = ['Acute fracture', 'Other', 'Pleural abnormality', 'Atelectasis', 'Pulmonary edema', 'Consolidation', 'Enlarged cardiac silhouette', 'Pneumothorax', 'Enlarged hilum', 'Interstitial lung disease', 'Groundglass opacity', 'Abnormal mediastinal contour', 'Lung nodule or mass', 'Pleural effusion', 'Fracture', 'Pleural thickening', 'High lung volume / emphysema', 'Hiatal hernia', 'Airway wall thickening', 'Emphysema', 'Nodule', 'Wide mediastinum', 'Fibrosis', 'Mass', 'Nothing']
# calculator for scalene triangle: https://www.omnicalculator.com/math/scalene-triangle
# visual angle and distance to size: https://elvers.us/perception/visualAngle/#:~:text=The%20visual%20angle%20of%20an,lead%20to%20larger%20visual%20angles.


# 71cm to top of screen, 68cm to the bottom, angle of 2º distance in screen 6.71cm
# screen has 64ppcm => 429.44 pixels of diameter => ~215 pixels of radius
#RADIUS = 215

# using the visual angle calculator
RADIUS = 90.5472  # assuming distance to the screen of 64.84 using the scalene formula
#RADIUS = 94.96    # assuming distance to the screen of 68 using the mean of top and bottom

with open("/content/drive/MyDrive/REFLACX-Study/Pickle/reflacx_dirs.pkl",'rb') as file:
  reflacx = pickle.load(file)

with open("/content/drive/MyDrive/REFLACX-Study/Pickle/fixations.pkl",'rb') as file:
  #reflacx_fixations {id: [normal_fixs, silence_fixs, ms_fixs], ...}
  #fixations is array [x,y,raw_time,timestamp]
  reflacx_fixations = pickle.load(file)

In [ ]:
#@title Study the relation between time of the report and number of clusters
#%%script echo skipping
reflacx_gaze = {}

def graphRelation(key, sname, pname, size_x, size_y, image_path, fixations_path):
  try:
    fixations = pd.read_csv(fixations_path)
  except FileNotFoundError:
    return 0
  
  global id
  transc_start = reflacx_fixations[id][2][0][-1]

  sample = fixations
  # initial fixations
  init = fixations[ (fixations.xmin_shown_from_image == 0) & (fixations.ymin_shown_from_image == 0)].head(1)
  IMG_W = size_x 
  IMG_H = size_y
  # get initial screeen coordinates
  screen_xmin = init.xmin_in_screen_coordinates.values[0]
  screen_xmax = init.xmax_in_screen_coordinates.values[0]
  screen_ymin = init.ymin_in_screen_coordinates.values[0]
  screen_ymax = init.ymax_in_screen_coordinates.values[0]

  # we can now determine what the screen's dimensions are in the image space
  xmax_image = int(np.round((SCREEN_RES[0]*(IMG_W))/(SCREEN_RES[0]-(screen_xmin)),0))
  left_menu_Xsize_img = int(np.round((screen_xmin*(IMG_W))/(SCREEN_RES[0]-(screen_xmin)),0))
  # in the video demo provided with the paper, the right menu has the same size as the left menu
  # however, in terms of data, the full right menu is cropped
  # we will mimic the video demo and assume the right menu = the left menu
  right_menu_Xsize_img = left_menu_Xsize_img

  x_left = [-1*(left_menu_Xsize_img-1), -1]
  x_right = [IMG_W+1, IMG_W+1+right_menu_Xsize_img]
  
  space_points = []
  ms_space_points = []
  silence_space_points = []
  
  for (timestamp_sample, x_img, y_img) in zip( sample["timestamp_sample"].values, sample["x_position"].values, sample["y_position"].values):

      if x_left[0] < x_img < x_left[1] or x_right[0] < x_img < x_right[0]: 
        pass
      else:
        #if timestamp_sample >= transc_start:
         # ms_space_points.append([x_img,y_img,timestamp_sample])
        #else:
         # silence_space_points.append([x_img,y_img,timestamp_sample])

        space_points.append([x_img,y_img,timestamp_sample])

  reflacx_gaze[id] = [space_points,silence_space_points,ms_space_points]
  id += 1


In [ ]:
REFLACX = 1
IMAGE = 0
RADIUS = 94.96
id = 501
def someF():
  for key, value in reflacx.items():
    if key.startswith('patient'):
      image = f"{rootdir}/{key}/CXR-JPG/{value[IMAGE][0]}/{value[IMAGE][1]}"
      for reflacx_report in value[REFLACX]:
        if reflacx_report.startswith('P'):
          fixations = f"{rootdir}/{key}/REFLACX/{reflacx_report}/gaze.csv"
          graphRelation(key, value[IMAGE][0], reflacx_report, value[IMAGE][2], value[IMAGE][3], image, fixations)
          #if id == 1001:
           # return 0

someF()